In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import requests
from datetime import datetime


In [5]:
# Initialize Dash app
app = dash.Dash(__name__)

# Create layout
app.layout = html.Div([
    # Graph component
    dcc.Graph(id='live-graph'),

    # Triggers periodically
    dcc.Interval(
        id='interval-component',
        interval=10_000,
        n_intervals=0
    )
])

# Initialize empty dataframe
df = pd.DataFrame(columns=['timestamp', 'value'])


# Define callback to update graph
@app.callback(
    Output('live-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):  # increment from intervals

    # Request API
    box_id = "6645db6eeb5aad0007a6e4b6"
    url = f"https://api.opensensemap.org/boxes/{box_id}?format=json"
    data = requests.get(url).json()
    value = data['sensors'][1]['lastMeasurement']['value']
    timestamp = datetime.now()

    global df
    df = pd.concat([df, pd.DataFrame([{'timestamp': timestamp, 'value': float(value)}])], ignore_index=True)

    # Plot data
    trace = go.Scatter(x=df['timestamp'], y=df['value'], mode='lines+markers')
    layout = go.Layout(title='Real-time Sensor Data', xaxis=dict(title='Timestamp'), yaxis=dict(title='Value'))
    fig = go.Figure(data=[trace], layout=layout)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
